In [1]:
import numpy as np
import pandas as pd
#pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_columns', 100)
from bokeh.plotting import figure, show, output_file
from bokeh.models import ColumnDataSource, Quad
from datetime import datetime
from math import pi
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, recall_score, precision_score
import random
import matplotlib.pyplot as plt
import seaborn as sns
from time import time
t = time()

In [2]:
# Keras imports
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.convolutional import Convolution2D, MaxPooling2D, Conv1D, MaxPooling1D, AveragePooling1D, UpSampling1D
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional, TimeDistributed
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LearningRateScheduler, ModelCheckpoint
from keras.optimizers import Adam, RMSprop
from keras.utils import np_utils, generic_utils
from sklearn.metrics import f1_score
from keras.models import Model
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints import maxnorm
from keras import regularizers
from keras.utils.np_utils import to_categorical
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, Callback, ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.layers.advanced_activations import LeakyReLU
from weightnorm import data_based_init, AdamWithWeightnorm
from keras.utils import np_utils
import keras.backend as K
from itertools import product
from functools import partial
from keras.layers.noise import GaussianNoise
from keras.layers.advanced_activations import ELU
from keras.constraints import maxnorm
from numpy.random import seed
from sklearn.preprocessing import minmax_scale
from keras.optimizers import SGD
from keras.models import Sequential
from keras.layers import Dense, Flatten, Input, merge, add
from keras.models import load_model

Using TensorFlow backend.


In [3]:
from sklearn import model_selection
from sklearn.preprocessing import Imputer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import average_precision_score
from sklearn import cross_validation
from sklearn.model_selection import KFold
from matplotlib import pyplot
from sklearn.decomposition import PCA
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GroupKFold

/usr/local/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
%matplotlib inline
%load_ext autoreload
%autoreload 1

In [5]:
# Import custom scripts and functions
%aimport model_performance_evaluation
%aimport custom_metrics
%aimport data_processing
%aimport make_keras_generators

In [6]:
# !wget https://dl.dropboxusercontent.com/u/53977633/data/df072717611pm.csv
# !wget https://dl.dropboxusercontent.com/u/53977633/data/X072717611pm.csv
# !wget https://dl.dropboxusercontent.com/u/53977633/data/y072717611pm.csv

In [7]:
# X = pd.read_csv('/output/X072717611pm.csv')
# X.drop(X.columns[0], inplace=True, axis=1)
# y = pd.read_csv('/output/y072717611pm.csv')
# y.drop(y.columns[0], inplace=True, axis=1)
# df_ohlc_and_rectangles = pd.read_csv('/output/df072717611pm.csv')
# df_ohlc_and_rectangles.drop(df_ohlc_and_rectangles.columns[0], inplace=True, axis=1)

In [8]:
# Create list of target csvs
data = ["EURJPY1440.csv", "EURGBP1440.csv","USDCAD1440.csv","GBPUSD1440.csv",\
        "EURUSD14402.csv","USDCHF1440.csv","EURAUD14402.csv","AUDUSD1440.csv",\
               "GBPJPY1440.csv", "NZDUSD1440.csv", "EURUSD240.csv", 
#         'indi_EURCAD_15.csv', 
#        'indi_USDCAD_15.csv','indi_USDJPY_15.csv', 'indi_USDJPY_60.csv' 
       ]

# allows us to easily modify how many days ahead and before we look:
lookforward = 3 # how many days we look forward
lookback = 7 # how many days we look back

        
# Choose features to be used in feature generation function
feature_generation_channels = ["O", "H", "L", "C"] 

X, y, df_ohlc_and_rectangles, groups,\
groups_count, scalers, groups_dict,\
scaled_columns, df_data_unscaled,\
y_rects, proximal_columns, distal_columns = data_processing.prepare_data(data, 
                                                                 lookforward, 
                                                                 lookback,
                                                                 feature_generation_channels,
                                                                 lag_data=1,
                                                                 change_open_close=1, 
                                                                 range_high_low=1,
                                                                 rolling_mean_range_HL=1,
                                                                 range_high_close=0,
                                                                 std_dev=1,
                                                                 rolling_mean=1,
                                                                 change_open_close_shift=0,
                                                                 range_high_low_shift=0,
                                                                 range_high_close_shift=0,
                                                                 diff_shift=1,
                                                                 rolling_mean_shift=1,
                                                                 std_dev_shift=0,
                                                                 center=False
                                                                )

Processing:  EURJPY1440.csv Number  1  of  11
We now have: (0, 0) rows of data!
Reading in Date, OHLCV, Supply/Demand
Renaming df_ohlc column headers for clarity
Reading in Rectangle data
Renaming df_rects column headers for clarity
Identifying supply/demand with supply_or_demand func: EURJPY1440.csv (4950, 12)
~~~
Identifying zone ends with zone_ender func: EURJPY1440.csv (4950, 14)
~~~
Your df_norm columns: Index(['O', 'H', 'L', 'C', 'V'], dtype='object')
Your df_non_norm columns: Index(['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start',
       'Epoch Date End', 'Proximal', 'Distal', 'supply_demand',
       'rectangle_here', 'Proximal_match_-3_O', 'Proximal_match_-2_O',
       'Proximal_match_-1_O', 'Proximal_match_0_O', 'Proximal_match_1_O',
       'Proximal_match_2_O', 'Proximal_match_3_O', 'Proximal_match_4_O',
       'Proximal_match_5_O', 'Proximal_match_6_O', 'Distal_match_-3_O',
       'Distal_match_-2_O', 'Distal_match_-1_O', 'Distal_match_0_O',
       'Distal_m

/output/data_processing.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_data["change_open_close"] = df_data["O"] - df_data["C"] # add change column, difference between open and close
/output/data_processing.py:60: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_data["range_high_low"] = df_data["H"] - df_data["L"] # add range, difference between high and low
/output/data_processing.py:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instea

Generating features:  Generic lagged data
Generating features:  Diffs over time for lookforward/lookback range
Generating features:  Rolling mean for yesterday, today, tomorrow range
Your df_scale_with_PCA: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12']
Your df_scale_with_PCA columns are: ['O', 'H', 'L', 'C', 'V', 'change_open_close', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_d

Identifying supply/demand with supply_or_demand func: USDCAD1440.csv (4943, 12)
~~~
Identifying zone ends with zone_ender func: USDCAD1440.csv (4943, 14)
~~~
Your df_norm columns: Index(['O', 'H', 'L', 'C', 'V'], dtype='object')
Your df_non_norm columns: Index(['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start',
       'Epoch Date End', 'Proximal', 'Distal', 'supply_demand',
       'rectangle_here', 'Proximal_match_-3_O', 'Proximal_match_-2_O',
       'Proximal_match_-1_O', 'Proximal_match_0_O', 'Proximal_match_1_O',
       'Proximal_match_2_O', 'Proximal_match_3_O', 'Proximal_match_4_O',
       'Proximal_match_5_O', 'Proximal_match_6_O', 'Distal_match_-3_O',
       'Distal_match_-2_O', 'Distal_match_-1_O', 'Distal_match_0_O',
       'Distal_match_1_O', 'Distal_match_2_O', 'Distal_match_3_O',
       'Distal_match_4_O', 'Distal_match_5_O', 'Distal_match_6_O',
       'Proximal_match_-3_H', 'Proximal_match_-2_H', 'Proximal_match_-1_H',
       'Proximal_match_0_H', 'Proximal_

Generating features:  Generic lagged data
Generating features:  Diffs over time for lookforward/lookback range
Generating features:  Rolling mean for yesterday, today, tomorrow range
Your df_scale_with_PCA: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12']
Your df_scale_with_PCA columns are: ['O', 'H', 'L', 'C', 'V', 'change_open_close', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_d

We now have: (24309, 232) rows of data!
Reading in Date, OHLCV, Supply/Demand
Renaming df_ohlc column headers for clarity
Reading in Rectangle data
Renaming df_rects column headers for clarity
Identifying supply/demand with supply_or_demand func: USDCHF1440.csv (4887, 12)
~~~
Identifying zone ends with zone_ender func: USDCHF1440.csv (4887, 14)
~~~
Your df_norm columns: Index(['O', 'H', 'L', 'C', 'V'], dtype='object')
Your df_non_norm columns: Index(['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start',
       'Epoch Date End', 'Proximal', 'Distal', 'supply_demand',
       'rectangle_here', 'Proximal_match_-3_O', 'Proximal_match_-2_O',
       'Proximal_match_-1_O', 'Proximal_match_0_O', 'Proximal_match_1_O',
       'Proximal_match_2_O', 'Proximal_match_3_O', 'Proximal_match_4_O',
       'Proximal_match_5_O', 'Proximal_match_6_O', 'Distal_match_-3_O',
       'Distal_match_-2_O', 'Distal_match_-1_O', 'Distal_match_0_O',
       'Distal_match_1_O', 'Distal_match_2_O', 'Distal_m

Generating features:  Generic lagged data
Generating features:  Diffs over time for lookforward/lookback range
Generating features:  Rolling mean for yesterday, today, tomorrow range
Your df_scale_with_PCA: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12']
Your df_scale_with_PCA columns are: ['O', 'H', 'L', 'C', 'V', 'change_open_close', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_d

Finished: AUDUSD1440.csv (4951, 232)
--------------------------------
Processing:  GBPJPY1440.csv Number  9  of  11
We now have: (38771, 232) rows of data!
Reading in Date, OHLCV, Supply/Demand
Renaming df_ohlc column headers for clarity
Reading in Rectangle data
Renaming df_rects column headers for clarity
Identifying supply/demand with supply_or_demand func: GBPJPY1440.csv (4978, 12)
~~~
Identifying zone ends with zone_ender func: GBPJPY1440.csv (4978, 14)
~~~
Your df_norm columns: Index(['O', 'H', 'L', 'C', 'V'], dtype='object')
Your df_non_norm columns: Index(['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start',
       'Epoch Date End', 'Proximal', 'Distal', 'supply_demand',
       'rectangle_here', 'Proximal_match_-3_O', 'Proximal_match_-2_O',
       'Proximal_match_-1_O', 'Proximal_match_0_O', 'Proximal_match_1_O',
       'Proximal_match_2_O', 'Proximal_match_3_O', 'Proximal_match_4_O',
       'Proximal_match_5_O', 'Proximal_match_6_O', 'Distal_match_-3_O',
       'D

Generating features:  Generic lagged data
Generating features:  Diffs over time for lookforward/lookback range
Generating features:  Rolling mean for yesterday, today, tomorrow range
Your df_scale_with_PCA: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11', 'PCA12']
Your df_scale_with_PCA columns are: ['O', 'H', 'L', 'C', 'V', 'change_open_close', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_d

Merging the scaled data and the unscalable data back together with pd.concat: EURUSD240.csv (8749, 232)
Finished: EURUSD240.csv (8749, 232)
--------------------------------
Finished loading data! You'll probably still want to remove some columns though


In [9]:
# Save data into backup X, y so we don't have to reload it 

X_backup = X
y_backup = y

In [10]:
X = X_backup
y = y_backup

In [11]:
labels_distal = [col for col in list(y) if col.startswith('Distal_')]
labels_proximal = [col for col in list(y) if col.startswith('Proximal_')]
labels_rectangle_here = [col for col in list(y) if col.startswith('rectangle_')]

In [12]:
# Make list of features and label column names
features = [col for col in list(X)]
labels = labels_proximal
reduced_match_list = ['Proximal_match_1_C', 
                      'Proximal_match_0_C', 
                      'Proximal_match_0_O', 
                      'Proximal_match_-1_O']

# Select only rows that contain zones, eliminate minority class elements (only about 4% of data)
df_zones = df_ohlc_and_rectangles.loc[df_ohlc_and_rectangles['rectangle_here'] == 1]
df_zones['sum'] = df_zones[reduced_match_list].sum(axis=1)
df_zones = df_zones[df_zones['sum'] != 0]

# Split out the new df into features and labels
df_X = df_zones[features]
df_y = df_zones[reduced_match_list]

# Drop features from X
features_to_remove = ['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start', 'L_diff_0', 'O_diff_0',
                      'Epoch Date End', 'Proximal', 'Distal', 'change', 'V_diff_0', 'C_diff_0', 'H_diff_0',
                      'O', 'H', 'L', 'C', 
                      'csv_origin_tag', 'range_high_close_range_0', 'change_open_close', 'range_high_close', 
                      'rectangle_here', 'supply_demand', 'group']

df_X = data_processing.drop_non_features(df_X, features_to_remove, zero_columns=1)
df_y = data_processing.drop_non_features(df_y, ['Proximal'], zero_columns=1)

# Clean up duplicate labels (i.e. where multiple matches were found)
df_y = df_y.apply(lambda x:x.drop_duplicates(), axis=1).fillna(False)


print(df_y.shape)
print(df_X.shape)

/usr/local/lib/python3.5/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


(7316, 4)
(7316, 129)


In [13]:
# Resort the DF and add what we think are the more important features to the front of the columns list
# Also, this removes all the plain 'lag' data, which we think has no meaning
X_cols_resort = ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]

df_X = df_X[X_cols_resort]

In [14]:
# Create autoencoded features
X_ae = data_processing.autoencode(df_X, df_y)

88


/output/data_processing.py:865: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  autoencoder = Model(input = input_dim, output = decoded4)


Train on 3292 samples, validate on 366 samples
Epoch 1/20
3292/3292 [==============================] - 0s - loss: 0.6926 - val_loss: 0.6923
Epoch 2/20
3292/3292 [==============================] - 0s - loss: 0.6921 - val_loss: 0.6918
Epoch 3/20
3292/3292 [==============================] - 0s - loss: 0.6916 - val_loss: 0.6913
Epoch 4/20
3292/3292 [==============================] - 0s - loss: 0.6911 - val_loss: 0.6908
Epoch 5/20
3292/3292 [==============================] - 0s - loss: 0.6906 - val_loss: 0.6903
Epoch 6/20
3292/3292 [==============================] - 0s - loss: 0.6900 - val_loss: 0.6897
Epoch 7/20
3292/3292 [==============================] - 0s - loss: 0.6894 - val_loss: 0.6890
Epoch 8/20
3292/3292 [==============================] - 0s - loss: 0.6888 - val_loss: 0.6883
Epoch 9/20
3292/3292 [==============================] - 0s - loss: 0.6880 - val_loss: 0.6875
Epoch 10/20
3292/3292 [==============================] - 0s - loss: 0.6872 - val_loss: 0.6866
Epoch 11/20
3292/3292 

/output/data_processing.py:870: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
  encoder = Model(input = input_dim, output = encoded4)


(7316, 3)
Original n of features: 88
autoencoded_features: 3


In [15]:
# Add together the main features and the autoencoder features
X_ = pd.concat([df_X.reset_index(drop=True), X_ae.reset_index(drop=True)], axis=1, ignore_index=True)  
X_.columns = list(df_X) + list(X_ae)

In [16]:
# Create a list to re-sort with again, putting our AE features at the front
X__cols_resort = list(X_ae) + ['change_open_close_diff_-1',  'C_rolling_mean_diff_-1', 'O_rolling_mean_diff_1', 'H_diff_1', 'C_diff_1', 'change_open_close_diff_1', 'H_diff_2', 'C_diff_2', 'change_open_close_diff_2', 'H_diff_3', 'C_diff_3','V', 'range_high_low', 'O_std_dev', 'H_std_dev', 'L_std_dev', 'C_std_dev', 'O_rolling_mean', 'H_rolling_mean', 'L_rolling_mean', 'C_rolling_mean', 'change_open_close_rolling_mean', 'rolling_mean_range_HL', 'O_diff_-3', 'H_diff_-3', 'L_diff_-3', 'C_diff_-3', 'change_open_close_diff_-3', 'O_diff_-2', 'H_diff_-2', 'L_diff_-2', 'C_diff_-2', 'change_open_close_diff_-2', 'O_diff_-1', 'H_diff_-1', 'L_diff_-1', 'C_diff_-1', 'change_open_close_diff_3', 'O_diff_4', 'H_diff_4', 'L_diff_4', 'C_diff_4', 'change_open_close_diff_4', 'O_diff_5', 'H_diff_5', 'L_diff_5', 'C_diff_5', 'change_open_close_diff_5', 'O_diff_6', 'H_diff_6', 'L_diff_6', 'C_diff_6', 'change_open_close_diff_6', 'O_rolling_mean_diff_-3', 'H_rolling_mean_diff_-3', 'L_rolling_mean_diff_-3', 'C_rolling_mean_diff_-3', 'O_rolling_mean_diff_-2', 'H_rolling_mean_diff_-2', 'L_rolling_mean_diff_-2', 'C_rolling_mean_diff_-2', 'O_rolling_mean_diff_-1', 'H_rolling_mean_diff_-1', 'L_rolling_mean_diff_-1', 'H_rolling_mean_diff_1', 'L_rolling_mean_diff_1', 'C_rolling_mean_diff_1', 'O_rolling_mean_diff_2', 'H_rolling_mean_diff_2', 'L_rolling_mean_diff_2', 'C_rolling_mean_diff_2', 'O_diff_1','L_diff_1','O_diff_2','L_diff_2','O_diff_3','L_diff_3', 'PCA1', 'PCA2', 'PCA3', 'PCA4', 'PCA5', 'PCA6', 'PCA7', 'PCA8', 'PCA9', 'PCA10', 'PCA11',]
X_ = X_[X__cols_resort]

In [17]:
# Create train/test split
X_train, X_test, y_train, y_test = train_test_split(X_, df_y, test_size=0.1)

# Create validation split from train split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.05)

In [18]:
# Stack original X_train to 3dim for conv net
X_train = data_processing.X_to_Conv1D_arrays(X_train)
X_valid = data_processing.X_to_Conv1D_arrays(X_valid)
X_test = data_processing.X_to_Conv1D_arrays(X_test)

X:  (6254, 91, 1)
X:  <class 'numpy.ndarray'>
X:  (330, 91, 1)
X:  <class 'numpy.ndarray'>
X:  (732, 91, 1)
X:  <class 'numpy.ndarray'>


In [19]:
print (X_train.shape, X_valid.shape, X_test.shape, y_train.shape, y_valid.shape, y_test.shape)

(6254, 91, 1) (330, 91, 1) (732, 91, 1) (6254, 4) (330, 4) (732, 4)


### LSTM for Proximal_match

In [20]:
# Create generators and define batch_size and other hyperparams
batch_size = 64

# Create generators
train_gen = make_keras_generators.train_generator(X_train, y_train.values, batch_size)
valid_gen = make_keras_generators.valid_generator(X_valid, y_valid.values, batch_size)

input_shape = X_train.shape[1:3]
output_shape = df_y.shape[1]
validation_steps = int(len(X_valid)/batch_size)+1
steps_per_epoch = int(len(X_train)/batch_size)+1
epochs = 15

validation_steps = int(len(X_valid)/batch_size)+1

In [21]:
mc_filepath = 'proximal-AE-july6-325pm-LSTM-{epoch:02d}-{val_fbeta_score:.2f}.h5'
mc = ModelCheckpoint(mc_filepath, 
                     monitor='val_fbeta_score', 
                     verbose=1, 
                     #save_weights_only=True,
                     period=10, mode='max',
                     save_best_only=False)

rlop = ReduceLROnPlateau(
                    monitor='val_loss', 
                    mode='min',
                    factor=0.6,
                    patience=10, 
                    min_lr=0.00001,
                    verbose=1,
                        )

es = EarlyStopping(
        monitor='val_loss',
        #min_delta=0.01,
        mode='min',
        patience=50,
        verbose=1)



callbacks = [ es, rlop, mc,]

### Define the proximal_match model

In [ ]:
#Use this cell if we want to start from scratch
proximal_model = Sequential()
proximal_model.add(Bidirectional(LSTM(128, return_sequences=False,
                                     recurrent_dropout=0.3, dropout=0.3), 
                                 input_shape=input_shape, 
                                 batch_input_shape=(None, X_train.shape[1], X_train.shape[2])))
proximal_model.add(Dropout(0.3))
proximal_model.add(Dense(output_shape, activation='softmax'))

In [22]:
proximal_model = load_model('proximal-AE-july6-131pm-LSTM-83-0.84.h5', custom_objects={'AdamWithWeightnorm': AdamWithWeightnorm, 'fbeta_score': custom_metrics.fbeta_score, 'recall': custom_metrics.recall, 'precision': custom_metrics.precision})

In [ ]:
proximal_model.compile(optimizer=AdamWithWeightnorm(0.0025), 
              loss='categorical_crossentropy',
                metrics = ['accuracy', 
                           custom_metrics.fbeta_score,
                           custom_metrics.recall,
                          custom_metrics.precision,]
            )


history = proximal_model.fit_generator(
                    train_gen,
                    steps_per_epoch=int(len(X_train)/batch_size)+1,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data = valid_gen,
                    #class_weight = class_weight,
                    validation_steps = validation_steps
                   )

In [ ]:
proximal_model.save('proximal-LSTM-07.06.17.324pm.h5')

In [25]:
preds, y_pred_proba = model_performance_evaluation.eval(proximal_model, 
                                                        X_test, 
                                                        y_test.values,
                                                        #weights='rectangle_here-Conv1D-19-0.61.h5',
                                                        batch_size=512, temperature=0.65, binary=0, sequential=0)

Your classification report: 
             precision    recall  f1-score   support

          0       0.00      0.00      0.00         0
          1       0.97      0.92      0.95       347
          2       0.79      0.93      0.86       321
          3       0.33      0.12      0.18        64

avg / total       0.84      0.86      0.84       732

-----------------------------
Log loss: 0.322061272654
F1 weighted: 0.840573676009
F1 macro: 0.496426650273
F1 micro: 0.857923497268
Accuracy score: 0.857923497268
Hamming Loss: 0.07103825136612021


/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1115: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [24]:
sns.set_context("paper", font_scale=2)
plt.figure(figsize=(12, 12))
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['fbeta_score'])
plt.plot(history.history['val_fbeta_score'])
plt.title('fbeta_score')
plt.ylabel('fbeta_score')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()


# summarize history for loss
fig_loss = plt.figure(figsize=(10, 10))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

# show lr changing
sns.set_context("paper", font_scale=2)
plt.figure(figsize=(12, 12))
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['lr'])
#plt.plot(history.history['val_fbeta_score'])
plt.title('learning rate')
plt.ylabel('lr')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

NameError: name 'history' is not defined

# Make predictions on raw data and visualize

In [ ]:
data = ['floyd-ignore/USDCAD1440.csv']

# allows us to easily modify how many days ahead and before we look:
lookforward = 6 # how many days we look forward
lookback = 6 # how many days we look back

        
# Choose features to be used in feature generation function
feature_generation_channels = ["O", "H", "L", "C", "V"] 

X, y, df_ohlc_and_rectangles, groups,\
groups_count, scalers, groups_dict,\
scaled_columns, df_data_unscaled,\
y_rects, proximal_columns, distal_columns = data_processing.prepare_data(data, 
                                                                 lookforward, 
                                                                 lookback,
                                                                 feature_generation_channels,
                                                                 lag_data=1,
                                                                 change_open_close=1, 
                                                                 range_high_low=1,
                                                                 rolling_mean_range_HL=1,
                                                                 range_high_close=1,
                                                                 std_dev=1,
                                                                 rolling_mean=1,
                                                                 change_open_close_shift=1,
                                                                 range_high_low_shift=0,
                                                                 range_high_close_shift=1,
                                                                 diff_shift=1,
                                                                 rolling_mean_shift=1,
                                                                 std_dev_shift=1,
                                                                 center=True
                                                                )

In [ ]:
# Make list of features and label column names
features = [col for col in list(X)]
labels_distal = [col for col in list(df_ohlc_and_rectangles) if col.startswith('Distal_')]
labels_proximal = [col for col in list(df_ohlc_and_rectangles) if col.startswith('Proximal_')]
labels = labels_distal + labels_proximal

# Select only rows that contain zones
#df_zones = df_ohlc_and_rectangles.loc[df_ohlc_and_rectangles['rectangle_here'] == 1]
df_ohlc = df_ohlc_and_rectangles

# Split out the new df into features and labels
#df_X = df_zones[features]
#df_y = df_zones[labels]
df_X = df_ohlc_and_rectangles[features]
df_y_proximal = df_ohlc_and_rectangles[labels_proximal]
df_y_distal = df_ohlc_and_rectangles[labels_distal]


%aimport data_processing

# Drop features from X
features_to_remove = ['Epoch Date', 'Supply/Demand', 'Object Name', 'Epoch Date Start', 'L_diff_0', 'O_diff_0',
                      'Epoch Date End', 'Proximal', 'Distal', 'change', 'V_diff_0', 'C_diff_0', 'H_diff_0',
                      #'O', 'H', 'L', 'C', 
                      'csv_origin_tag', 'range_high_close_range_0', 'change_open_close', 'range_high_close', 
                      #'rectangle_here', 
                      'supply_demand', 'group']


df_X = data_processing.drop_non_features(df_X, features_to_remove, zero_columns=1)
df_y_proximal = data_processing.drop_non_features(df_y_proximal, ['Proximal'], zero_columns=1)
df_y_distal = data_processing.drop_non_features(df_y_distal, ['Distal'], zero_columns=1)

df_y_proximal = df_y_proximal.apply(lambda x:x.drop_duplicates(), axis=1).fillna(False)
df_y_distal = df_y_distal.apply(lambda x:x.drop_duplicates(), axis=1).fillna(False)

In [ ]:
lag_cols = [col for col in list(df_X) if col.startswith('lag')]
ohlc_cols = ['O', 'H', 'L', 'C', 'V'] + lag_cols

df_X_ohlc = df_X[ohlc_cols]

In [ ]:
# For lining up predictions and data
#df_predictions = pd.DataFrame(preds.astype(int))
df_y = pd.concat([df_y_proximal, 
                          df_y_distal], 
                         axis=1, 
                         ignore_index=True)  
df_y.columns = list(df_y_proximal) + list(df_y_distal)

In [ ]:
# Add together the raw ohlc data and the y labels
df_OHLC = pd.concat([df_X_ohlc, 
                          df_y], 
                         axis=1, 
                         ignore_index=True)  
df_OHLC.columns = list(df_X_ohlc) + list(df_y)

In [ ]:
# Add the rectangle_here columns back on 
df_OHLC_ = pd.concat([pd.DataFrame(df_ohlc_and_rectangles['rectangle_here']).reset_index(drop=True), 
                          df_OHLC], 
                         axis=1, 
                         ignore_index=True)  
df_OHLC_.columns = list(pd.DataFrame(df_ohlc_and_rectangles['rectangle_here'])) + list(df_OHLC)

In [ ]:
def proximal_distal_match_invert(df):
    
    # Set all Supply/Demand row to nans to start clean
    df['Proximal_discrete_value'] = np.nan
    df['Distal_discrete_value'] = np.nan
    
    # Create list of all zones, filtered by if there's a potential zone in the row
    potential_zone = df[df.rectangle_here.isnull() == False]
    
    # Create list of the indices of the zones
    potential_zone_indices = potential_zone.index.tolist()
            
    labels_distal = [col for col in list(df) if col.startswith('Distal')]
    labels_proximal = [col for col in list(df) if col.startswith('Proximal')]
    
    # For each zone
    for idx in potential_zone_indices:
        #print("Processing index: ", idx)
        # If current iterable index + 1 is > len (e.g. if we are at the end of the DF)
        if idx+1 > len(df):

            # Quit the loop
            break
        
            
        for distal_match_col in labels_distal:
            #print(df.iloc[idx][distal_match_col])
            #print(distal_match_col)
            if df.ix[idx, distal_match_col] == True:
                lag_col_match = "lag_" + distal_match_col[13:]
                #print('Lag col match is:', lag_col_match)
                #print('The value contained at idx/lag_col_match: ', idx, ", ", df.iloc[idx][lag_col_match])
                #print('The value at idx/distal_discrete_value: ', idx, ", ", df.iloc[idx]['Distal_discrete_value'])
                distal_discrete_value = df.iloc[idx][lag_col_match]
                df.ix[idx, 'Distal_discrete_value'] = distal_discrete_value
                #print('The value at idx/distal_discrete_value: ', idx, ", ", df.iloc[idx]['Distal_discrete_value'])
    
        for proximal_match_col in labels_proximal:
            #print(df.iloc[idx][distal_match_col])
            #print(distal_match_col)
            if df.ix[idx, proximal_match_col] == True:
                lag_col_match = "lag_" + proximal_match_col[15:]
                #print('Lag col match is:', lag_col_match)
                #print('The value contained at idx/lag_col_match: ', idx, ", ", df.iloc[idx][lag_col_match])
                #print('The value at idx/proximal_discrete_value: ', idx, ", ", df.iloc[idx]['Proximal_discrete_value'])
                proximal_discrete_value = df.iloc[idx][lag_col_match]
                df.ix[idx, 'Proximal_discrete_value'] = proximal_discrete_value
                #print('The value at idx/proximal_discrete_value: ', idx, ", ", df.iloc[idx]['Proximal_discrete_value'])



In [ ]:
proximal_distal_match_invert(df_OHLC_)

In [ ]:
list(df_ohlc_and_rectangles)

## Visualize preds and OHLC

In [ ]:
import zigzag as zigzag

In [ ]:
ohlc_data_col_list = ['O',
 'H',
 'L',
 'C',
 'V',
 'Proximal_discrete_value',
 'Distal_discrete_value']

df_ohlc_and_rectangles_col_list = ['Epoch Date',
 'Supply/Demand',
 'Epoch Date Start',
 'Epoch Date End',]

new_ohlc_data_col_list = ['O',
 'H',
 'L',
 'C',
 'V',
 'Proximal',
 'Distal']

In [ ]:
# Add the rectangle_here columns back on 
df_ohlc_data = pd.concat([df_OHLC_[ohlc_data_col_list], 
                          df_ohlc_and_rectangles[df_ohlc_and_rectangles_col_list]], 
                         axis=1, 
                         ignore_index=True)

df_ohlc_data.columns = new_ohlc_data_col_list + df_ohlc_and_rectangles_col_list

In [ ]:
df_ohlc_data['Epoch Date End'] = np.nan

In [ ]:
data_processing.zone_ender(df_ohlc_data)

In [ ]:
data_processing.supply_or_demand(df_ohlc_data)

In [ ]:
def viz_ohlc(df):    
    # Convert Dates to datetime
    df["Epoch Date"] = pd.to_datetime(df["Epoch Date"], unit='s')
    df["Epoch Date Start"] = pd.to_datetime(df["Epoch Date Start"], unit='s')
    df["Epoch Date End"] = pd.to_datetime(df["Epoch Date End"], unit='s')

    w = 12*60*60*1000 # half day in ms

    TOOLS = "pan,xwheel_zoom,wheel_zoom,box_zoom,reset,save"
    p = figure(x_axis_type="datetime", tools=TOOLS, plot_height=500, 
               plot_width=1200, title="🍩", active_scroll='xwheel_zoom')
    p.xaxis.major_label_orientation = pi/4
    p.grid.grid_line_alpha=0.5

    p.segment(df['Epoch Date'], df.H, df['Epoch Date'], df.L, color="black")
    p.vbar(df['Epoch Date'], w, df.O, df.C)

    # Add Rectangle data
    df_sp = df.dropna(subset=['Supply/Demand'])

    source_supply = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='SUPPLY', :])
    rect_supply = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="firebrick", line_alpha=0.2, fill_alpha=0.2) 
    p.add_glyph(source_supply, rect_supply)

    source_demand = ColumnDataSource(df_sp.loc[df_sp['Supply/Demand']=='DEMAND', :])
    rect_demand = Quad(left="Epoch Date Start", top="Proximal", right="Epoch Date End", bottom="Distal", fill_color="steelblue", line_alpha=0.2, fill_alpha=0.2)
    p.add_glyph(source_demand, rect_demand)


    output_file("candlestick.html", title="candlestick data")
    show(p)  # open a browser

In [ ]:
viz_ohlc(df_ohlc_data)